<a href="https://colab.research.google.com/github/dhanushpandian/GroQ_LLM/blob/main/GROQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain-groq
!pip install langchain_community langchain
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from getpass import getpass
hf_token=getpass("enter hugging face access token")
groq_api=getpass("GroQ API")


enter hugging face access token··········
GroQ API key··········


In [24]:
URLS=[
    "https://www.psychologytoday.com/us/basics/psychology",
    "https://www.verywellmind.com/psychology-basics-4157186",
    "https://www.ibm.com/topics/sentiment-analysis",
    "https://www.geeksforgeeks.org/what-is-sentiment-analysis/"
]


In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import Chroma

In [25]:
loader = WebBaseLoader(URLS)
data=loader.load()

In [26]:
data

[Document(page_content='\n\n\nPsychology | Psychology Today\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Skip to main content\n    \n\n\n\n\n\n\nMobile Navigation\n\nPsychology Today\n\n\nFind a Therapist\n\n\n\n\nFind a Therapist\n\n\n\n\n\n\n\nTherapists\n\n\n\nTherapists\n\n\nPsychiatrists\n\n\nTreatment Centers\n\n\nSupport Groups\n\n\n\n\nx\n\n\n\n\n\n\n\n\n\n\n\n            Therapists\n      :\nLogin\n|\nSign Up\n\n\n\n\nUnited States\n\nAustin, TX\nBrooklyn, NY\nChicago, IL\nDenver, CO\nHouston, TX\nLos Angeles, CA\nNew York, NY\nPortland, OR\nSan Diego, CA\nSan Francisco, CA\nSeattle, WA\nWashington, DC\nAtlanta, GA\nSacramento, CA\n\n\n\n\n\n\n\nGet Help\n\n\n\n\n\n\nMental Health\n\n\nAddiction\nAnxiety\nADHD\nAsperger\'s\nAutism\nBipolar Disorder\nChronic Pain\nDepression\nEating Disorders\n\n\n\n\n\n\nPersonality\n\n\nPassive Ag

In [27]:
text_splitter =RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=150)
website_data= text_splitter.split_documents(data)

In [28]:
embeddings= HuggingFaceInferenceAPIEmbeddings(api_key=hf_token,model_name="WhereIsAI/UAE-Large-V1")

vectorstore= Chroma.from_documents(website_data,embeddings,persist_directory="db")


In [15]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

In [31]:
retriever = vectorstore.as_retriever()

template = """
User: You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

Keep in mind, you will lose the job, if you answer out of CONTEXT questions

CONTEXT: {context}
Query: {question}

Remember only return AI answer
Assistant:
"""

In [29]:
chat = ChatGroq(temperature=0.1, groq_api_key=groq_api, model="mixtral-8x7b-32768")

In [32]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | chat
    | output_parser
)

In [33]:
query="What is sentiment analysis?"
for chunk in chain.stream(query):
  print(chunk, end="", flush=True)

Sentiment analysis is a process in natural language processing that classifies the text based on the mood or mentality expressed. It can be positive, negative, or neutral. The goal is to analyze people’s opinions in a way that can help businesses expand. It focuses not only on polarity but also on emotions. It uses various Natural Language Processing algorithms such as Rule-based, Automatic, and Hybrid.